# Setup

Source: https://github.com/samwit/langchain-tutorials/blob/main/YT_Langchain_with_DuckDuckGo_Wikipedia_PythonREPL_Template.ipynb

In [ ]:
# Loads .env variables
%load_ext dotenv
%dotenv

## Wikipedia

In [6]:
from langchain.utilities import WikipediaAPIWrapper

In [7]:
wikipedia = WikipediaAPIWrapper()

In [5]:
wikipedia.run('rhodolith')

'Page: Rhodolith\nSummary: Rhodoliths (from Greek for red rocks) are colorful, unattached calcareous nodules, composed of crustose, benthic marine red algae that resemble coral. Rhodolith beds create biogenic habitat for diverse benthic communities. The rhodolithic growth habit has been attained by a number of unrelated coralline red algae, organisms that deposit calcium carbonate within their cell walls to form hard structures or nodules that resemble beds of coral. \nRhodoliths do not attach themselves to the rocky seabed. Rather, they roll like tumbleweeds along the seafloor until they become too large in size to be mobilised by the prevailing wave and current regime. They may then become incorporated into a semi-continuous algal mat or form an algal build-up. While corals are animals that are both autotrophic (photosynthesize via their symbionts) or heterotrophic (feeding on plankton), rhodoliths produce energy solely through photosynthesis (i.e. they can only grow and survive in t

## REPL

In [8]:
from langchain.utilities import PythonREPL

In [9]:
python_repl = PythonREPL()

In [8]:
python_repl.run("print(17*2)")

Python REPL can execute arbitrary code. Use with caution.


'34\n'

## DuckDuckGo

In [10]:
from langchain.tools import DuckDuckGoSearchRun

In [11]:
search = DuckDuckGoSearchRun()

In [17]:
search.run("rhodolith beds")

"Rhodolith. Characteristic mauve coloured rhodolith. Rhodoliths (from Greek for red rocks) are colorful, unattached calcareous nodules, composed of crustose, benthic marine red algae that resemble coral. Rhodolith beds create biogenic habitat for diverse benthic communities. The rhodolithic growth habit has been attained by a number of unrelated ... Rhodolith beds harbor a highly diverse associated biota. The Brazilian coast encompasses the world's largest rhodolith beds of the world that occur a Biogeographic patterns of marine fishes associated with rhodolith beds in the Southwestern Atlantic reveal an ecotone of biodiversity | SpringerLink Skip to main content Advertisement Log in Menu Rhodolith beds stand with kelp beds, seagrass meadows, and coralline algal reefs as one of the world's four largest macrophyte-dominated benthic communities. Geographic distribution of rhodolith beds is discontinuous, with large concentrations off Japan, Australia and the Gulf of California, as well a

# Putting them together

In [12]:
from langchain import OpenAI
from langchain.tools import Tool

llm = OpenAI(temperature=0)

In [13]:
repl_tool = Tool(
    name = "Python REPL",
    func = python_repl.run,
    description = "useful for when you need to use python to answer a question. You should input python code"
)

wikipedia_tool = Tool(
    name='Wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

tools = [repl_tool, wikipedia_tool, duckduckgo_tool]

# Using the Agents

We'll use the tools with a Zero Shot Agent

In [14]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [15]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Python REPL: useful for when you need to use python to answer a question. You should input python code
Wikipedia: Useful for when you need to look up a topic, country or person on wikipedia
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python REPL, Wikipedia, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [16]:
zero_shot_agent.run("When was Barak Obama born?")



> Entering new AgentExecutor chain...
 I need to find out when Barak Obama was born.
Action: Wikipedia
Action Input: Barak Obama
Observation: Page: Barack Obama
Summary: Barack Hussein Obama II (  bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and worked as a civil rights lawyer and university lecturer. 
Obama was born in Honolulu, Hawaii. He attended Occidental College from 1979 to 1981 and transferred to finished his undergraduate studies at Columbia University from 1981 to 1983. After graduating from Columbia University in 1983, he worked as a community organizer in Chicago. In 1988, he enrolled in Harvard Law School, where he was the first black president of the Harvard 

'Barack Obama was born on August 4, 1961.'

In [30]:
zero_shot_agent.run("What is 17*6?")



> Entering new AgentExecutor chain...
 I need to do a calculation
Action: Python REPL
Action Input: 17*6
Observation: 
Thought: I now know the final answer
Final Answer: 102

> Finished chain.


'102'

In [32]:
zero_shot_agent.run("Tell me about LangChain")



> Entering new AgentExecutor chain...
 I should look up LangChain on the internet
Action: DuckDuckGo Search
Action Input: "LangChain"
Observation: 🦜️🔗 LangChain. ⚡ Building applications with LLMs through composability ⚡. Looking for the JS/TS version? Check out LangChain.js.. Production Support: As you move your LangChains into production, we'd love to offer more hands-on support. Fill out this form to share more about what you're building, and our team will get in touch.. 🚨Breaking Changes for select chains (SQLDatabase ... LangChain is an SDK designed to simplify the integration of LLMs and applications. It solves most of the challenges that we discussed above. LangChain is similar to an ODBC or JDBC driver, which ... LangChain is an open-source Python library that enables anyone who can write code to build LLM-powered applications. The package provides a generic interface to many foundation models, enables prompt management, and acts as a central interface to other components like

'LangChain is an open-source Python library that enables anyone who can write code to build LLM-powered applications. It includes API wrappers, web scraping subsystems, code analysis tools, document summarization tools, and more. It also supports large language models and offers a suite of tools, components, and interfaces that simplify the process of creating applications powered by large language models (LLMs) and chat models.'

In [34]:
zero_shot_agent.run("Tell me about the country Brazil")



> Entering new AgentExecutor chain...
 I should look up Brazil on Wikipedia
Action: Wikipedia
Action Input: Brazil
Observation: Page: Brazil
Summary: Brazil (Portuguese: Brasil; Brazilian Portuguese: [bɾaˈziw] ), officially the Federative Republic of Brazil (Portuguese: ), is the largest country in South America and in Latin America. Brazil is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the only country in the Americas to have Portuguese as an official language. It is one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world, and the most populous Roman Catholic-majority country.
Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi). It borders all other countries and territories in South Ameri

"Brazil is the largest country in South America and in Latin America. It is the world's fifth-largest country by area and the seventh most populous. It has Portuguese as an official language and is one of the most multicultural and ethnically diverse nations. It is also home to the Brazil men's national football team, which is administered by the Brazilian Football Confederation."

In [35]:
zero_shot_agent.run('Write a function to check if 11 a prime number and test it')



> Entering new AgentExecutor chain...
 I need to write a function to check if a number is prime
Action: Python REPL
Action Input: 
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
Observation: 
Thought: I need to test the function
Action: Python REPL
Action Input: 
print(is_prime(11))
Observation: True

Thought: I now know the final answer
Final Answer: 11 is a prime number.

> Finished chain.


'11 is a prime number.'

In [36]:
zero_shot_agent.run('Explain to me the main aspects of a rhodolith bed')



> Entering new AgentExecutor chain...
 I need to research this topic
Action: Wikipedia
Action Input: Rhodolith bed
Observation: Page: Rhodolith
Summary: Rhodoliths (from Greek for red rocks) are colorful, unattached calcareous nodules, composed of crustose, benthic marine red algae that resemble coral. Rhodolith beds create biogenic habitat for diverse benthic communities. The rhodolithic growth habit has been attained by a number of unrelated coralline red algae, organisms that deposit calcium carbonate within their cell walls to form hard structures or nodules that resemble beds of coral. 
Rhodoliths do not attach themselves to the rocky seabed. Rather, they roll like tumbleweeds along the seafloor until they become too large in size to be mobilised by the prevailing wave and current regime. They may then become incorporated into a semi-continuous algal mat or form an algal build-up. While corals are animals that are both autotrophic (photosynthesize via their symbionts) or heterot

"Rhodolith beds are composed of crustose, benthic marine red algae that resemble coral. They create biogenic habitat for diverse benthic communities and do not attach themselves to the rocky seabed. They have been present in the world's oceans since at least the Eocene epoch, some 55 million years ago. Maerl is a collective name for non-geniculate coralline red algae with a certain growth habit and Laminaria abyssalis is a species of brown kelp, notable for its connection to rhodolith beds in the Brazilian coastline."